В этом задании вам предлагается начать разбираться с задачей анализа тональности отзывов на примере сентимент-анализа отзывов на фильмы.

In [1]:
from nltk.corpus import movie_reviews
import nltk
import numpy as np

In [2]:
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Цифра-2\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [3]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [4]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]


# Week 1

1. Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.


In [5]:
all_feats = negfeats + posfeats
classes = np.append(np.zeros(len(negfeats)), np.ones(len(posfeats)))

2. Подсчитайте количество отзывов в выборке.


In [6]:
ans_1 = classes.shape[0]
print(ans_1)

2000


3. Подсчитайте долю класса 1 в выборке


In [7]:
ans_2 = len(posfeats) / ans_1
print(ans_2)

0.5


4. Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
all_str = list(map(lambda x: " ".join(x), all_feats))

In [10]:
count_vec = CountVectorizer()
vect_repr = count_vec.fit_transform(all_str)

In [11]:
vect_repr

<2000x39659 sparse matrix of type '<class 'numpy.int64'>'
	with 666842 stored elements in Compressed Sparse Row format>

In [12]:
ans_3 = 39659

5. Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [14]:
pipe = Pipeline([
    ('count_vect', CountVectorizer()),
    ('log_reg', LogisticRegression())
])

In [15]:
ans_4 = np.mean(cross_val_score(pipe, all_str, classes, cv=3))
print(ans_4)

0.8360216503929078


6. Аналогично accuracy, оцените качество по ROC AUC

In [16]:
ans_5 = np.mean(cross_val_score(pipe, all_str, classes, cv=3, scoring='roc_auc'))
print(ans_5)

0.9107764937833774


7. Обучите логистическую регрессию на всей доступной вам выборке и выведите два наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.


In [17]:
c_vect = CountVectorizer().fit(all_str)
cls = LogisticRegression().fit(c_vect.transform(all_str), classes)

In [18]:
coeffs = cls.coef_[0]


In [19]:
sorted(coeffs)[:2], sorted(coeffs)[-2:] 

([-0.7822544710446759, -0.6366239501417396],
 [0.44416042537462447, 0.5560506991496287])

In [20]:
top_2 = sorted(coeffs)[:2]
ans_6 = " ".join([c_vect.get_feature_names()[list(coeffs).index(i)] for i in top_2])
ans_6

'bad unfortunately'

# Week 2

1. Здесь и далее оценка качества будет выполняться с помощью cross_val_score с cv=5 и остальными параметрами по умолчанию. Оцените среднее качество (.mean()) и стандартное отклонение (.std()) по fold'ам для:

    а) pipeline из CountVectorizer() и LogisticRegression(), 
    б) pipeline из TfidfVectorizer() и LogisticRegression(). 

В соответствующем пункте задания выпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
def write_ans(ans, n_task):
    with open('answer_' + str(n_task) + '.txt', 'w') as f:
        f.write(str(ans))

In [23]:
pipe_cv_lr_1 = Pipeline([
    ('count_vect', CountVectorizer()),
    ('log_reg', LogisticRegression())
])

pipe_tf_lr_1 = Pipeline([
    ('count_vect', TfidfVectorizer()),
    ('log_reg', LogisticRegression())
])

In [24]:
cv = 5

a_ans = cross_val_score(pipe_cv_lr_1, all_str, classes, cv=cv)
b_ans = cross_val_score(pipe_tf_lr_1, all_str, classes, cv=cv)

ans = (a_ans.mean(), a_ans.std(), b_ans.mean(), b_ans.std())
ans_2_1 = " ".join([str(i) for i in ans])
write_ans(ans_2_1, 1)
print('Средние значения и стандартные отклонинея для CountVectorizer, TfidfVectorizer:', ans_2_1)

Средние значения и стандартные отклонинея для CountVectorizer, TfidfVectorizer: 0.8415000000000001 0.01677796173556255 0.8210000000000001 0.004062019202317978


2. Попробуйте задавать разные значения параметра min_df у CountVectorizer. Оцените качество вашего классификатора с min_df=10 и с min_df=50.

In [25]:
ans = np.array([])
for i in [10, 50]:
    pipe_cv_lr_2 = Pipeline([
        ('count_vect', CountVectorizer(min_df=i)),
        ('log_reg', LogisticRegression())
    ])
    ans = np.append(ans, np.mean(
        cross_val_score(pipe_cv_lr_2, all_str, classes, cv=cv)
    ))
    
ans_2_2 = " ".join([str(i) for i in ans])

write_ans(ans_2_2, 2)
print('Качество работы при разных параметрах min_df CountVectorizer:', ans_2_2)

Качество работы при разных параметрах min_df CountVectorizer: 0.8390000000000001 0.813


3. Попробуйте использовать разные классификаторы после CountVectorizer. И vectorizer, и классификатор берите с параметрами по умолчанию. Сравните результаты для LogisticRegression, LinearSVC и SGDClassifier. Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся.


In [26]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

import warnings
warnings.filterwarnings("ignore")

In [28]:
ans = []
for i in ([LogisticRegression(), LinearSVC(), SGDClassifier(random_state=0)]):
    pipe_cv_lr_3 = Pipeline([
        ('count_vect', CountVectorizer()),
        ('log_reg', i)
    ])
    ans = np.append(ans, np.mean(cross_val_score(pipe_cv_lr_3, all_str, classes, cv=cv)))

ans_2_3 = min(ans)
write_ans(ans_2_3, 3)
print('Качество работы худшего из классификаторов (LogisticRegression, LinearSVC, SGDClassifier):',
      ans_2_3)

Качество работы худшего из классификаторов (LogisticRegression, LinearSVC, SGDClassifier): 0.7004999999999999


4. Подготовьте список стоп-слов с помощью nltk.corpus.stopwords.words('english'), посмотрите на его элементы, и передайте его в соответствующий параметр CountVectorizer. В sklearn также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе.

In [29]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Цифра-2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
ans = []
for i in [CountVectorizer(stop_words=stop_words), CountVectorizer(stop_words='english')]:
    pipe_cv_lr_4 = Pipeline([
        ('count_vect', i),
        ('log_reg', LogisticRegression())
    ])
    ans = np.append(ans, np.mean(cross_val_score(pipe_cv_lr_4, all_str, classes, cv=cv)))

ans_2_4 = " ".join([str(i) for i in ans])
write_ans(ans_2_4, 4)
print('Качество работы CountVectorizer при использовании стоп-слов:', ans_2_4)


Качество работы CountVectorizer при использовании стоп-слов: 0.841 0.8385


5. Попробуйте в CountVectorizer добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра ngram_range и параметр analyzer='char_wb'. Полученные два числа запишите через пробел в ответе на соответствующий вопрос.

In [31]:
ans = []
for i in [CountVectorizer(ngram_range=(1, 2)), CountVectorizer(ngram_range=(3,5), analyzer='char_wb')]:
    pipe_cv_lr_5 = Pipeline([
        ('count_vect', i),
        ('log_reg', LogisticRegression())
    ])
    ans = np.append(ans, np.mean(cross_val_score(pipe_cv_lr_5, all_str, classes, cv=cv)))

ans_2_5 = " ".join([str(i) for i in ans])
write_ans(ans_2_5, 5)
print('Качество работы CountVectorizer при униграммах и биграммах и буквенных n-граммах:', ans_2_5)

Качество работы CountVectorizer при униграммах и биграммах и буквенных n-граммах: 0.8525 0.82
